In [9]:
import numpy as np
from tabulate import tabulate
import random

wallSymbol = str('\U00002B1B')
floorSymbol = str('\U00002B1C')
botSymbol = str('\U0001F916')
buttonSymbol = str('\U0001F7E5')
fireSymbol = str('\U0001F525')

D = 10 #Ship dimensions variable
q = 0.5

def theShip(D, opencells):
    options = []
    ship = np.full((D, D), int(0), dtype = object)
    X, Y = random.randrange(D), random.randrange(D) #Selects random point on ship for first cell to open
    opencells.append(tuple([X, Y]))

    markOpen(ship, X, Y, D, options) 
    #markOpen will open the given XY and increase each closed neighbor C by +1, adding them to the options list IF and ONLY IF C == 1, meaning C only has one opened neighbor
    while len(options) >= 1:
        #while there are available XY in options, this will iterate continuously
        rand = random.randrange(len(options))
        X, Y = options[rand]
        opencells.append(tuple([X, Y]))
        options = markOpen(ship, X, Y, D, options)

    #print(tabulate(ship))  IGNORE
    #print(options)         IGNORE

    deadends = []
    for X in range(D): #This code chunk iterates over every cell on the ship to figure out which are dead ends (have only one side that is an X). 
        for Y in range(D):
            if isinstance(ship[X][Y], int):
                ship[X][Y] = wallSymbol
            if isinstance(ship[X][Y], str):
                isDeadend = 0
                if X - 1 >= 0:
                    if ship[X - 1][Y] == floorSymbol:
                        isDeadend += 1
                if X + 1 < D:
                    if ship[X + 1][Y] == floorSymbol:
                        isDeadend += 1
                if Y - 1 >= 0:
                    if ship[X][Y - 1] == floorSymbol:
                        isDeadend += 1
                if Y + 1 < D:
                    if ship[X][Y + 1] == floorSymbol:
                        isDeadend += 1
                if isDeadend == 1: #If isDeadend is 1, then this cell only has one opened neighbor and is a deadend
                    deadends.append(tuple([X, Y]))
    random.shuffle(deadends) #Randomizes dead end list so that we can randomly select the first half of them
         
    #print(tabulate(ship))
    #print(deadends)

    for i in range(int(len(deadends) / 2)): #The first half of the dead ends list is iterated on, this code chunk figures out how many neighbors are available to open
        X, Y = deadends[i]
        randselect = []
        randnum = 0
        if Y - 1 >= 0:
            if isinstance(ship[X][Y - 1], int):
                randselect.append(ship[X][Y - 1])
                randnum += 1 #randnum is used in the next code chunk
        if X - 1 >= 0:
            if isinstance(ship[X - 1][Y], int):
                randselect.append(ship[X - 1][Y])
                randnum += 1
        if X + 1 < D:
            if isinstance(ship[X + 1][Y], int):
                randselect.append(ship[X + 1][Y])
                randnum += 1
        if Y + 1 < D:
            if isinstance(ship[X][Y + 1], int):
                randselect.append(ship[X][Y + 1])
                randnum += 1
        
        if randnum > 0: #This code chunk uses randnum to 'randomly' select one of the closed neighbors to open, then removes XY from dead end list
            randnum = random.randrange(randnum) 
            deadends.remove(tuple([X, Y]))
            if Y - 1 >= 0:
                if isinstance(ship[X][Y - 1], int):
                    if randnum == 0:
                        ship[X][Y - 1] = str(floorSymbol)
                        opencells.append(tuple([X, Y - 1]))
                    randnum -= 1
            if X - 1 >= 0:
                if isinstance(ship[X - 1][Y], int):
                    if randnum == 0:
                        ship[X - 1][Y] = str(floorSymbol)
                        opencells.append(tuple([X - 1, Y]))
                    randnum -= 1
            if X + 1 < D:
                if isinstance(ship[X + 1][Y], int):
                    if randnum == 0:
                        ship[X + 1][Y] = str(floorSymbol)
                        opencells.append(tuple([X + 1, Y]))
                    randnum -= 1
            if Y + 1 < D:
                if isinstance(ship[X][Y + 1], int):
                    if randnum == 0:
                        ship[X][Y + 1] = str(floorSymbol)
                        opencells.append(tuple([X, Y + 1]))
                    randnum -= 1

    print(tabulate(ship))

    print(deadends)

    return opencells, ship

def markOpen(ship, X, Y, D, options):
    ship[X][Y] = str(floorSymbol) 
    
    if (X, Y) in options: 
        options.remove(tuple([X, Y])) #Remove current XY from list of available closed cells as its being opened

    if Y - 1 >= 0: #Checks south of current XY for valid open cells
        if isinstance(ship[X][Y - 1], int):
            ship[X][Y - 1] += 1 #At any given point, closed cells should display how many opened neighbors they have, from 1 to 4, courtesy of this command
            if ship[X][Y - 1] == 1:
                options.append(tuple([X, Y - 1]))
            elif ship[X][Y - 1] > 1 and (X, Y - 1) in options:
                options.remove(tuple([X, Y - 1]))
    if X - 1 >= 0: #Checks west of current XY for valid open cells
        if isinstance(ship[X - 1][Y], int):
            ship[X - 1][Y] += 1 #At any given point, closed cells should display how many opened neighbors they have, from 1 to 4, courtesy of this command
            if ship[X - 1][Y] == 1:
                options.append(tuple([X - 1, Y]))
            elif ship[X - 1][Y] > 1 and (X - 1, Y) in options:
                options.remove(tuple([X - 1, Y]))
    if X + 1 < D: #Checks east of current XY for valid open cells
        if isinstance(ship[X + 1][Y], int):
            ship[X + 1][Y] += 1 #At any given point, closed cells should display how many opened neighbors they have, from 1 to 4, courtesy of this command
            if ship[X + 1][Y] == 1:
                options.append(tuple([X + 1, Y]))
            elif ship[X + 1][Y] > 1 and (X + 1, Y) in options:
                options.remove(tuple([X + 1, Y]))
    if Y + 1 < D: #Checks north of current XY for valid open cells
        if isinstance(ship[X][Y + 1], int):
            ship[X][Y + 1] += 1 #At any given point, closed cells should display how many opened neighbors they have, from 1 to 4, courtesy of this command
            if ship[X][Y + 1] == 1:
                options.append(tuple([X, Y + 1]))
            elif ship[X][Y + 1] > 1 and (X, Y + 1) in options:
                options.remove(tuple([X, Y + 1])) 
    
    #print(tabulate(ship))     IGNORE
    #print(options)            IGNORE
    
    return options

def placeObjects(bot, button, fire0, ship, opencells, fireNeighbors):
    ship[bot[0]][bot[1]] = botSymbol
    ship[button[0]][button[1]] = buttonSymbol
    fireNeighbors.append(tuple([fire0[0], fire0[1]]))
    fireNeighbors, opencells = startFire(fire0, ship, opencells, fireNeighbors)
    print(tabulate(ship))
    return fireNeighbors, opencells

def spreadFire(fireNeighbors, opencells, ship):
    toFire = []

    for i in fireNeighbors:
        K = 0
        if i[0] - 1 >= 0:
            if ship[i[0] - 1][i[1]] == fireSymbol:
                K += 1
        if i[0] + 1 < D:
            if ship[i[0] + 1][i[1]] == fireSymbol:
                K += 1
        if i[1] - 1 >= 0:
            if ship[i[0]][i[1] - 1] == fireSymbol:
                K += 1
        if i[1] + 1 < D:
            if ship[i[0]][i[1] + 1] == fireSymbol:
                K += 1
        print(K)
        p = (1 - ((1 - q) ** K))
        print(p)
        if random.uniform(0, 1) <= p:
            toFire.append(tuple(i))
            print(toFire)
    
    for i in toFire:
        startFire(i, ship, opencells, fireNeighbors)

    return opencells, fireNeighbors

def startFire(Fire, ship, opencells, fireNeighbors):
    ship[Fire[0]][Fire[1]] = fireSymbol
    fireNeighbors.remove(tuple([Fire[0], Fire[1]]))
    opencells.remove(tuple([Fire[0], Fire[1]]))
    if Fire[1] - 1 >= 0:
        if ship[Fire[0]][Fire[1] - 1] == floorSymbol or ship[Fire[0]][Fire[1] - 1] == botSymbol or ship[Fire[0]][Fire[1] - 1] == buttonSymbol:
            fireNeighbors.append(tuple([Fire[0], Fire[1] - 1]))
    if Fire[1] + 1 < D:
        if ship[Fire[0]][Fire[1] + 1] == floorSymbol or ship[Fire[0]][Fire[1] + 1] == botSymbol or ship[Fire[0]][Fire[1] + 1] == buttonSymbol:
            fireNeighbors.append(tuple([Fire[0], Fire[1] + 1]))
    if Fire[0] + 1 < D:
        if ship[Fire[0] + 1][Fire[1]] == floorSymbol or ship[Fire[0] + 1][Fire[1]] == botSymbol or ship[Fire[0] + 1][Fire[1]] == buttonSymbol:
            fireNeighbors.append(tuple([Fire[0] + 1, Fire[1]]))
    if Fire[0] - 1 >= 0:
        if ship[Fire[0] - 1][Fire[1]] == floorSymbol or ship[Fire[0] - 1][Fire[1]] == botSymbol or ship[Fire[0] - 1][Fire[1]] == buttonSymbol:
            fireNeighbors.append(tuple([Fire[0] - 1, Fire[1]]))
    '''print('fire neighbors') 
    print(fireNeighbors)
    print(len(opencells))'''
    return fireNeighbors, opencells
        
def main():
    t = 100
    ti = 0
    opencells = []
    opencells, ship = theShip(D, opencells)

    #print(opencells)

    #t = 0 begins here
    random.shuffle(opencells)
    print(len(opencells))

    fireNeighbors = []
    placeObjects(opencells[0], opencells[1], opencells[2], ship, opencells, fireNeighbors)

    while ti < t:
        ti += 1
        opencells, fireNeighbors = spreadFire(fireNeighbors, opencells, ship)
        print(tabulate(ship))


if __name__ == "__main__":
    main()

--  --  --  --  --  --  --  --  --  --
⬛  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛
⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛
⬜  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜
⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬛  ⬜  ⬛
⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜
⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜
⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜
⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛
⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜
⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜
--  --  --  --  --  --  --  --  --  --
[(4, 9), (1, 0), (0, 1), (9, 1), (9, 4), (6, 9), (2, 9), (0, 3), (7, 3), (8, 9), (0, 5), (1, 4), (5, 2), (3, 8), (8, 0), (7, 8), (6, 1), (0, 7)]
63
--  --  --  --  --  --  --  --  --  --
⬛  ⬜  🤖  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛
⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛
⬜  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜
⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬛  🟥  ⬛
⬜  ⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛  ⬜
⬜  ⬛  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜  ⬜  ⬜
⬜  ⬜  ⬛  ⬛  ⬜  🔥  ⬛  ⬜  ⬛  ⬜
⬜  ⬛  ⬛  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬛
⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  ⬛  ⬛  ⬜
⬛  ⬜  ⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜
--  --  --  --  --  --  --  --  --  --
1
0.5
[(6, 4)]
1
0.5
--  --  --  --  --  --  --  --  --  --
⬛  ⬜  🤖  ⬜  ⬛  ⬜  ⬛  ⬜  ⬛  ⬛
⬜  ⬛  ⬜  ⬛  ⬜  ⬜  ⬜  ⬜  ⬜  